In [50]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score,KFold

In [2]:
exercise=pd.read_csv("C:\\Users\\DELL\\Desktop\\Project 2 RE\\exercise.csv")
calories=pd.read_csv('C:\\Users\\DELL\\Desktop\\Project 2 RE\\calories.csv')

In [3]:
exercise.shape
# here we have imported the exercise data 

(15000, 8)

In [4]:
exercise.isnull().sum()

User_ID       0
Gender        0
Age           0
Height        0
Weight        5
Duration      0
Heart_Rate    0
Body_Temp     0
dtype: int64

In [5]:
exercise.sample(10)
# this will require one hot encoding for the nominal categorical column GENDER 
# I ALSO NEED TO USE SIMPLE IMPUTATION TO FILL NAN ENTRIES 
# ALSO IT WILL REQUIRE FEATURE SCALING AND I WILL TRY BOTH STANDARDSCALER AND MINMAXSCALER(NORAMLISATION)
# I SUSPECT HIGH MULTICOLINEARITY SO IT REQUIRES FEATURE EXTRACTION AND FEATURE SELECTION TOO
# THEN I WILL HAVE TO APPLY DIFFERENT TYPES OF ML MODELS AND CEHCK FOR ACCURACY USING R^2 AND ADUSTED R^2 OR MAE


,User_ID,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
6637,16768990,female,25,167,64.0,20,100,40.4
10292,15853624,male,27,203,98.0,24,90,40.7
8004,19214945,female,64,170,67.0,2,90,37.9
5555,14936913,male,20,190,93.0,16,99,40.2
11245,11300613,male,74,201,108.0,5,87,38.9
6865,19376524,male,57,186,89.0,9,83,39.6
2793,14254315,male,73,170,75.0,5,89,38.9
9928,12143206,male,49,184,91.0,14,96,40.1
1562,16323950,female,77,150,55.0,5,80,39.0
7328,18280227,male,22,194,92.0,17,97,40.3


In [6]:
calories.head()
# and here we have the correspodnding calories data and note that User_ID of both the columns sync well



,User_ID,Calories
0,14733363,231.0
1,14861698,66.0
2,11179863,26.0
3,16180408,71.0
4,17771927,35.0


In [7]:
# let us firstly combine both the dataframes
df=pd.concat([exercise,calories['Calories']],axis=1)


In [8]:
df.shape

(15000, 9)

In [9]:
df.head()

,User_ID,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,14733363,male,68,190,94.0,29,105,40.8,231.0
1,14861698,female,20,166,60.0,14,94,40.3,66.0
2,11179863,male,69,179,79.0,5,88,38.7,26.0
3,16180408,female,34,179,71.0,13,100,40.5,71.0
4,17771927,female,27,154,58.0,10,81,39.8,35.0


# EDA

In [10]:
# firstly we will drop columns which out insignificant for the model training 
df.drop(columns=['User_ID'],inplace=True)

In [11]:
df.head()
# so we have 8 columns now

,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,male,68,190,94.0,29,105,40.8,231.0
1,female,20,166,60.0,14,94,40.3,66.0
2,male,69,179,79.0,5,88,38.7,26.0
3,female,34,179,71.0,13,100,40.5,71.0
4,female,27,154,58.0,10,81,39.8,35.0


In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(df.iloc[:,0:7],df.iloc[:,7:8])

# FEATURE ENGINEERING

In [13]:
#simple imputer
#one hot encoding 
# let us do this with the help of column transformer 
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [14]:
transformer=ColumnTransformer(transformers=[('tnf2',OneHotEncoder(drop='first',sparse_output=False),['Gender']),('tnf1',SimpleImputer(),['Weight'])],remainder='passthrough')

In [15]:
# I WILL HAVE TO DO THREE THINGS 
# ONE HOT ENCODING 
# SCALING (STANDARDISATION OR NORMALISATION )
# SIMPLE IMPUTING   
X_train__=transformer.fit_transform(X_train)
X_test__=transformer.fit_transform(X_test)

In [16]:

X_train_=pd.DataFrame(X_train__,columns=['Gender','Weight','Age','Height','Duration','Heart_Rate','Body_Temp'])
X_test_=pd.DataFrame(X_test__,columns=['Gender','Weight','Age','Height','Duration','Heart_Rate','Body_Temp'])

In [17]:
X_test_.head()

,Gender,Weight,Age,Height,Duration,Heart_Rate,Body_Temp
0,0.0,66.0,47.0,157.0,22.0,96.0,40.4
1,1.0,86.0,35.0,190.0,8.0,93.0,39.5
2,0.0,56.0,45.0,158.0,11.0,87.0,39.7
3,0.0,75.0,27.0,181.0,10.0,89.0,39.6
4,1.0,82.0,48.0,179.0,11.0,90.0,39.6


In [18]:
# now we have our X_test_ and X_train_
# we have to scale this using Standardisation and also we have to look for multicolinearity before applying the model

# Standard Scaling

In [19]:
from sklearn.preprocessing import StandardScaler

In [20]:
ss=StandardScaler()
X_train=ss.fit_transform(X_train_)
X_test=ss.transform(X_test_)

In [21]:
X_train_f=pd.DataFrame(X_train,columns=['Gender','Weight','Age','Height','Duration','Heart_Rate','Body_Temp'])
X_test_f=pd.DataFrame(X_test,columns=['Gender','Weight','Age','Height','Duration','Heart_Rate','Body_Temp'])

In [22]:
X_train_f.head()

,Gender,Weight,Age,Height,Duration,Heart_Rate,Body_Temp
0,1.001423,0.263882,-1.098015,0.313174,-1.389615,-2.036138,-1.578531
1,-0.998579,-0.599474,1.132599,-0.247601,0.890970,0.773805,0.477964
2,1.001423,0.928003,1.250000,0.663659,0.050754,0.669733,0.349433
3,-0.998579,-0.665886,-0.511012,-0.387795,-1.629676,-1.411706,-2.863840
4,-0.998579,-0.732298,0.193393,-0.247601,-1.509646,-0.162843,-1.964124


In [23]:
# so finally after scaling we have 
X_train_f
X_test_f 
# now lets apply it on the model

,Gender,Weight,Age,Height,Duration,Heart_Rate,Body_Temp
0,-0.998579,-0.599474,0.252093,-1.228957,0.770939,0.045301,0.477964
1,1.001423,0.728767,-0.452311,1.084240,-0.909492,-0.266915,-0.678815
2,-0.998579,-1.263594,0.134693,-1.158860,-0.549399,-0.891346,-0.421753
3,-0.998579,-0.001766,-0.921914,0.453368,-0.669430,-0.683203,-0.550284
4,1.001423,0.463119,0.310794,0.313174,-0.549399,-0.579131,-0.550284
...,...,...,...,...,...,...,...
3745,1.001423,1.791359,-0.158809,1.504821,-1.269584,-0.475059,-1.192938
3746,1.001423,0.861591,0.134693,0.663659,-0.069276,0.149373,-0.036160
3747,1.001423,0.595943,-0.393611,0.453368,-1.509646,-1.932066,-1.835593
3748,-0.998579,-1.263594,-0.745813,-0.948570,-0.069276,-0.058771,-0.164691


In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_absolute_error

In [25]:
lr=LinearRegression()


In [26]:
lr.fit(X_train_f,Y_train)

LinearRegression()

In [27]:
Y_predict=lr.predict(X_test_f)

In [28]:
print(r2_score(Y_predict,Y_test))

0.9665069438314284


In [29]:
print(mean_absolute_error(Y_predict,Y_test))

8.265252450064084


In [30]:
def r2_adjusted(r):
    a=1-((1-r)*(3749))/(3743)
    return a

In [47]:
r2_adjusted(0.9665069438314284)

0.9664532547218876

In [32]:
# Let us cross validate it using cross validation techniques
print('avg:',np.mean(cross_val_score(lr,X_test_f,Y_test,cv=20)))

avg: 0.9670805257127706


# USING MIN MAX SCALER 

In [33]:
from sklearn.preprocessing import MinMaxScaler

In [34]:
mm=MinMaxScaler()

In [35]:
X_train_m=mm.fit_transform(X_train_)
X_test_m=mm.fit_transform(X_test_)

In [36]:
X_train_m

array([[1.        , 0.44791667, 0.06779661, ..., 0.10344828, 0.14754098,
        0.37209302],
       [0.        , 0.3125    , 0.71186441, ..., 0.75862069, 0.59016393,
        0.74418605],
       [1.        , 0.55208333, 0.74576271, ..., 0.51724138, 0.57377049,
        0.72093023],
       ...,
       [0.        , 0.36458333, 0.61016949, ..., 0.86206897, 0.44262295,
        0.79069767],
       [1.        , 0.54166667, 0.03389831, ..., 0.93103448, 0.63934426,
        0.8372093 ],
       [0.        , 0.125     , 0.06779661, ..., 0.5862069 , 0.39344262,
        0.76744186]])

In [37]:
X_train_f2=pd.DataFrame(X_train_m,columns=['Gender','Weight','Age','Height','Duration','Heart_Rate','Body_Temp'])
X_test_f2=pd.DataFrame(X_test_m,columns=['Gender','Weight','Age','Height','Duration','Heart_Rate','Body_Temp'])

In [38]:
X_train_f2.head()

,Gender,Weight,Age,Height,Duration,Heart_Rate,Body_Temp
0,1.0,0.447917,0.067797,0.547368,0.103448,0.147541,0.372093
1,0.0,0.312500,0.711864,0.463158,0.758621,0.590164,0.744186
2,1.0,0.552083,0.745763,0.600000,0.517241,0.573770,0.720930
3,0.0,0.302083,0.237288,0.442105,0.034483,0.245902,0.139535
4,0.0,0.291667,0.440678,0.463158,0.068966,0.442623,0.302326


In [39]:
# now we shall use this data for training our model 
lr2=LinearRegression()

In [40]:
lr2.fit(X_train_f2,Y_train)

LinearRegression()

In [41]:
Y_pred2=lr2.predict(X_test_f2)

In [48]:
print(mean_absolute_error(Y_pred2,Y_test))

8.875910606896847


In [42]:
a=r2_score(Y_pred2,Y_test)
a

0.966098125321932

In [43]:
def r2_adjusted(r):
    a=1-((1-r)*(3749))/(3743)
    return a

In [44]:
r2_adjusted(a)

0.9660437808794878

In [59]:
k_folds = KFold(n_splits = 5)
print('avg:',np.mean(cross_val_score(lr2,X_test_f2,Y_test,cv=k_folds)))

avg: 0.9672722290670528


In [46]:
# i can use more techniques
FUNCTION TRANSFORMER #to normalise the data 
                     # can also work with outliers to improve the performance of the data set


SyntaxError: invalid syntax (48129606.py, line 2)